In [1]:
from numpy import random
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import factorial
from PIL import Image
PATH = "img/"

In [2]:
sample_size = [10, 100, 1000]
loc, scale = 0.0, 1.0
EXP_NUM = 1000

In [3]:
def norm(size):
    return random.normal(loc, scale, size)

def cauchy(size):
    return random.standard_cauchy(size)

def laplace(size):
    return random.laplace(loc, 1 / np.sqrt(2), size)

def poisson(size):
    return random.poisson(10, size)

def uniform(size):
    return  random.uniform(-np.sqrt(3), np.sqrt(3), size)

distribution_fun = { 'norm' : norm,
                     'cauchy' : cauchy,
                     'laplace' : laplace,
                     'poisson' : poisson,
                     'uniform' : uniform, }


In [4]:
def calculate_sample_mean(sample):
     return np.mean(sample)
    
def calculate_median(sample):
     return np.median(sample)

def calculate_half_sum_extreme(sample):
    return (min(sample) + max(sample)) / 2

def calculate_quantile(sample, index):
    return np.quantile(sample, index)

def calculate_half_sum_quantile(sample):
    return (calculate_quantile(sample, 0.25) + calculate_quantile(sample, 0.75)) / 2



def calculate_truncated_mean(sample):
    res = 0
    n = len(sample)
    r = int(0.25 * n)
    i = r + 1
    while i <= n - r:
        res += sample[i]
        i = i + 1
    return res / (n - 2 * r)



characteristic = {
    'sample_mean': calculate_sample_mean,
    'median':calculate_median,
    'half_sum_extreme': calculate_half_sum_extreme,
    'half_sum_quantile': calculate_half_sum_quantile,
    'truncated_mean': calculate_truncated_mean,
}

In [5]:
def D(sample):
    return np.var(sample)

def E(sample):
    return np.mean(sample)

In [6]:
def calculate_mean_value(result):
    mean_value = {}
    answer = {
        'sample_mean': [],
        'median':[],
        'half_sum_extreme': [],
        'half_sum_quantile': [],
        'truncated_mean': [], }
    for ch in characteristic:
        mean_value['e']=E(result[ch])
        mean_value['d']=D(result[ch])
        answer[ch] = mean_value.copy()
    return answer

In [7]:
answer = {}
sample_res ={}
result ={}
for dist in distribution_fun:
    for size in sample_size:
        result.clear();
        result = {
        'sample_mean': [],
        'median':[],
        'half_sum_extreme': [],
        'half_sum_quantile': [],
        'truncated_mean': [], }
        for i in range(1000):
            sample = distribution_fun[dist](size)
            for ch in characteristic:
                tmp = characteristic[ch](sample)
                result[ch].append(tmp)  
        sample_res[str(size)]=calculate_mean_value(result)
    answer[dist] = sample_res.copy()
        

from pprint import pprint 
pprint(answer)


{'cauchy': {'10': {'half_sum_extreme': {'d': 11099.681872055064,
                                        'e': 1.1407574715773852},
                   'half_sum_quantile': {'d': 0.6470926379320494,
                                         'e': 0.06052868395901279},
                   'median': {'d': 0.3114225796656555,
                              'e': 0.013312177013279107},
                   'sample_mean': {'d': 449.03416407693715,
                                   'e': 0.28787105909809735},
                   'truncated_mean': {'d': 1146.4847303397205,
                                      'e': -0.0010740550992599082}},
            '100': {'half_sum_extreme': {'d': 5105926.712492397,
                                         'e': -76.65772235208055},
                    'half_sum_quantile': {'d': 0.055499236081479095,
                                          'e': -0.006742954272074258},
                    'median': {'d': 0.026502867020203134,
                               'e': -0

In [10]:
def show_results(dist, name):
    res_d =[]
    res_e=[]
    res_interval={'begin':[],'end':[]}
    res_i = []
    rows = []
    headers = [name , "x_", "med(x)", "z_R", "z_Q", "z_tr"]
    for size in dist:        
        res_d.clear()
        res_e.clear()
        res_interval['begin'].clear()
        res_interval['end'].clear()
        res_e.append(" E(z) =  " +str(size))
        res_d.append(" D(z) =  " +str(size))
        res_interval['begin'].append("E(z)-\sqrt(D(z))")
        res_interval['end'].append("E(z)+\sqrt(D(z))")
        for ch in dist[size]:
            res_d.append(np.around((dist[size])[ch]['d'],  decimals=4))
            res_e.append(np.around((dist[size])[ch]['e'],  decimals=4))
            res_interval['begin'].append(np.around((dist[size])[ch]['e']-np.sqrt((dist[size])[ch]['d']),  decimals=4))            
            res_interval['end'].append(np.around((dist[size])[ch]['e']+np.sqrt((dist[size])[ch]['d']),  decimals=4)) 
        rows.append(res_e.copy())        
        rows.append(res_d.copy())
        rows.append(["intervals","","","","",""])
        rows.append(res_interval['begin'].copy())
        rows.append(res_interval['end'].copy())
        rows.append(["","","","","",""])
    print(tabulate(rows, headers))
    
for key in distribution_fun:
    show_results(answer[key], key)
    print("\n")

norm              x_       med(x)    z_R      z_Q      z_tr
----------------  -------  --------  -------  -------  -------
E(z) =  10        0.0004   -0.0018   0.0017   0.0013   0.016
D(z) =  10        0.095    0.1326    0.1765   0.1092   0.163
intervals
E(z)-\sqrt(D(z))  -0.3078  -0.3659   -0.4184  -0.3291  -0.3876
E(z)+\sqrt(D(z))  0.3086   0.3624    0.4218   0.3317   0.4197

E(z) =  100       0.0035   0.0049    -0.0156  0.0038   0.0069
D(z) =  100       0.0104   0.0154    0.0919   0.012    0.0201
intervals
E(z)-\sqrt(D(z))  -0.0983  -0.1191   -0.3188  -0.1059  -0.1347
E(z)+\sqrt(D(z))  0.1053   0.1289    0.2876   0.1135   0.1486

E(z) =  1000      -0.0005  -0.0001   0.0168   -0.0013  0.0011
D(z) =  1000      0.001    0.0016    0.0618   0.0012   0.002
intervals
E(z)-\sqrt(D(z))  -0.0327  -0.0395   -0.2318  -0.0366  -0.0441
E(z)+\sqrt(D(z))  0.0317   0.0393    0.2653   0.034    0.0463



cauchy            x_         med(x)    z_R           z_Q      z_tr
----------------  ---------  --